In [196]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [197]:
df = pd.read_parquet(os.path.join('data','jizdenky.parquet'))
df[df['oscrapovano'] <= '2024-12-01']
df = df[df['predstih_h'] >= 0]

In [198]:
df['odjezd_dt'] = pd.to_datetime(df['odjezd'])
df = df[df['odjezd_dt'] <= '2024-12-14']

In [199]:
df.sort_values(by="odjezd",ascending=False)

,oscrapovano,prodejce,odkud,kam,odjezd,predstih,predstih_d,predstih_h,cena,prostredek,volnych_mist,obsazenost,jizdni_doba,vzdalenost,zpozdeni,cena_poznamka,den,prestupy,vlaky,mistenka_zdarma,nahradni_bus,volna_mista_economy,volna_mista_economy_plus,volna_mista_economy_business,volna_mista_premium,odjezd_dt
514908,2024-11-17 09:57:49,ČD,Wien Stadlau,Praha hl.n.,2024-12-13 23:58:00,26 days 14:00:11,26,638.003056,NaN,vlak,NaN,"[Předpokládané nízké obsazení, Téměr vyprodaná...",366.0,394.0,"[0, 0]",Zjistit cenu,pá,1,"[Ex 1136 Gepard Express/, R 876 Svitava]",None,None,NaN,NaN,NaN,NaN,2024-12-13 23:58:00
399675,2024-11-14 11:57:51,ČD,Wien Stadlau,Praha hl.n.,2024-12-13 23:58:00,29 days 12:00:09,29,708.002500,NaN,vlak,NaN,"[Předpokládané nízké obsazení, Téměr vyprodaná...",366.0,394.0,"[0, 0]",Zjistit cenu,pá,1,"[Ex 1136 Gepard Express/, R 876 Svitava]",None,None,NaN,NaN,NaN,NaN,2024-12-13 23:58:00
961250,2024-11-29 17:23:17,ARR,Pardubice hl.n.,Liberec,2024-12-13 23:57:00,14 days 06:33:43,14,342.561944,170.0,vlak,39.0,None,172.0,161.0,None,None,pá,0,[R 1076],None,None,NaN,NaN,NaN,NaN,2024-12-13 23:57:00
990459,2024-11-30 17:23:37,ARR,Pardubice hl.n.,Liberec,2024-12-13 23:57:00,13 days 06:33:23,13,318.556389,170.0,vlak,37.0,None,172.0,161.0,None,None,pá,0,[R 1076],None,None,NaN,NaN,NaN,NaN,2024-12-13 23:57:00
972978,2024-11-30 05:26:53,ARR,Pardubice hl.n.,Liberec,2024-12-13 23:57:00,13 days 18:30:07,13,330.501944,170.0,vlak,39.0,None,172.0,161.0,None,None,pá,0,[R 1076],None,None,NaN,NaN,NaN,NaN,2024-12-13 23:57:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142625,2024-11-08 07:50:23,RJ,Brno,Vídeň,2024-11-08 08:48:00,0 days 00:57:37,0,0.960278,399.0,vlak,172.0,None,93.0,NaN,None,None,pá,0,None,None,None,NaN,NaN,NaN,NaN,2024-11-08 08:48:00
143190,2024-11-08 07:53:06,RJ,Brno,Praha,2024-11-08 08:30:00,0 days 00:36:54,0,0.615000,548.0,vlak,9.0,None,217.0,NaN,None,None,pá,1,None,None,None,NaN,NaN,NaN,NaN,2024-11-08 08:30:00
143545,2024-11-08 08:03:15,ČD,Praha hl.n.,Ostrava hl.n.,2024-11-08 08:19:00,0 days 00:15:45,0,0.262500,619.0,vlak,NaN,[Předpokládané vysoké obsazení - doporučujeme ...,209.0,356.0,[],None,pá,0,[EC 141 Ostravan],True,None,NaN,NaN,NaN,NaN,2024-11-08 08:19:00
143177,2024-11-08 07:53:06,RJ,Brno,Praha,2024-11-08 08:09:00,0 days 00:15:54,0,0.265000,399.0,vlak,3.0,None,148.0,NaN,None,None,pá,0,None,None,None,NaN,NaN,NaN,NaN,2024-11-08 08:09:00


In [200]:
def filtr(dopravce, mesto1, mesto2, nasobek=1.25):
    dfc = df.copy()
    print(f"{dopravce}: {mesto1}-{mesto2}")
    dfc = dfc[dfc['prodejce'] == dopravce]
    dfc = dfc[dfc['odkud'].str.contains(mesto1) | dfc['kam'].str.contains(mesto1)]
    dfc = dfc[dfc['odkud'].str.contains(mesto2) | dfc['kam'].str.contains(mesto2)]
    minimum_prestupu = dfc['prestupy'].min()
    if minimum_prestupu == -1:
        print("Pozor, vylezlo nám tu -1 přestupů. Opravuji.")
        minimum_prestupu = 0
    print(f"Nejmenší počet přestupů: {minimum_prestupu}.")
    minimum_casu = dfc['jizdni_doba'].min()
    print(f"Nejrychlejší jízdní doba: {minimum_casu} min.")
    dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]
    dfc = dfc.drop_duplicates(subset=['odjezd','kam','predstih_d'], keep='last')
    print(f"Celkem řádků: {len(dfc)}\n")
    return dfc.reset_index(drop=True)

In [201]:
def obdobi(radek):
    return f"""{radek['odjezd'].dayofweek + 1}-{radek['odjezd'].hour // 4}"""

In [202]:
df['obdobi'] = df.apply(lambda row: obdobi(row), axis=1)

Střední cena jízdenky koupené do 24 hodin před odjezdem je na trase mezi Prahou a Brnem bez slev…

In [204]:
prabr = df[df['odkud'].str.contains('Praha') | df['odkud'].str.contains('Brno')]
print(len(prabr))
prabr = prabr[prabr['kam'].str.contains('Brno') | prabr['kam'].str.contains('Praha')]
print(len(prabr))
prabr = prabr[prabr['jizdni_doba'] <= 180]
print(len(prabr))
prabr = prabr[prabr['prestupy'] == 0]
print(len(prabr))
prabr = prabr[(prabr['odjezd'] >= '2024-11-11') & (prabr['odjezd'] <= '2024-12-1')]
print(len(prabr))
prabr = prabr[(prabr['predstih_h'] <= 24)]

349582
162895
124372
124309
86138


In [205]:
prabr['cena'].median()

359.0

In [206]:
df[df['odkud'].str.contains('Pra') & df['kam'].str.contains('Brn')].groupby('obdobi')['cena'].median().nlargest(10)

obdobi
6-0    449.0
5-0    435.0
5-2    435.0
5-3    435.0
5-4    435.0
7-0    419.0
1-0    409.0
6-2    399.0
7-3    399.0
2-0    359.0
Name: cena, dtype: float64

In [207]:
df[df['odkud'].str.contains('Brn') & df['kam'].str.contains('Pra')].groupby('obdobi')['cena'].median().nlargest(10)

obdobi
7-0    478.0
5-3    419.0
7-3    419.0
5-0    418.0
5-2    399.0
6-2    399.0
7-4    399.0
4-0    388.0
6-0    388.0
4-2    359.0
Name: cena, dtype: float64

In [208]:
df[df['odkud'].str.contains('Brn') & df['kam'].str.contains('Ostra')].groupby('obdobi')['cena'].median().nlargest(10)

obdobi
5-3    279.0
3-3    269.0
4-3    269.0
4-4    269.0
5-2    269.0
5-4    269.0
6-2    269.0
6-4    269.0
7-3    269.0
7-4    269.0
Name: cena, dtype: float64

In [209]:
df[df['odkud'].str.contains('Ostra') & df['kam'].str.contains('Brn')].groupby('obdobi')['cena'].median().nlargest(10)

obdobi
7-3    279.0
7-4    279.0
1-1    269.0
2-1    269.0
3-1    269.0
4-1    269.0
5-2    269.0
5-3    269.0
6-1    269.0
4-3    254.0
Name: cena, dtype: float64

In [210]:
df.groupby(["prodejce","odkud","kam"]).size().nlargest(200)

C:\Users\micha\AppData\Local\Temp\ipykernel_30048\1717840207.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["prodejce","odkud","kam"]).size().nlargest(200)


prodejce  odkud                             kam                          
ČD        Praha hl.n.                       Brno hl.n.                       54419
          Brno hl.n.                        Praha hl.n.                      53339
          Praha hl.n.                       Ostrava hl.n.                    41126
          Ostrava hl.n.                     Praha hl.n.                      39214
RJ        Brno                              Praha                            26924
          Praha                             Brno                             25051
          Ostrava                           Praha                            18134
ARR       Pardubice hl.n.                   Liberec                          16589
          Liberec                           Pardubice hl.n.                  16188
RJ        Praha                             Ostrava                          15284
ČD        Bratislava hl.st.                 Praha hl.n.                      10857
          Pra

In [211]:
cd_brno = filtr("ČD","Praha hl.n.","Brno hl.n.")
cd_brno = pd.Series(cd_brno.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Brno").head(30)
rj_brno = filtr("RJ","Praha","Brno")
rj_brno = pd.Series(rj_brno.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Brno").head(30)
rj_brnova = filtr("RJ","Brno","Ostrava")
rj_brnova = pd.Series(rj_brnova.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Brno-Ostrava").head(30)
arr_libpa = filtr("ARR","Pardubice","Liberec")
arr_libpa = pd.Series(arr_libpa.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ARR Liberec-Pardubice").head(30)
arr_bud = filtr("ARR","Praha","Budějovice")
arr_bud = pd.Series(arr_bud.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ARR Praha-Budějovice").head(30)
arr_libpa = filtr("ARR","Pardubice","Liberec")
arr_libpa = pd.Series(arr_libpa.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ARR Liberec-Pardubice").head(30)
cd_kv = filtr("ČD","Karlovy","Ostrava")
cd_kv = pd.Series(cd_kv.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Ostrava-Karlovy Vary").head(30)
cd_ova = filtr("ČD","Ostrava","Praha")
cd_ova = pd.Series(cd_ova.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Ostrava").head(30)
rj_ova = filtr("RJ","Ostrava","Praha")
rj_ova = pd.Series(rj_ova.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Ostrava").head(30)
le_ova = filtr("LE","Ostrava","Praha")
le_ova = pd.Series(le_ova.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="LE Praha-Ostrava").head(30)
ukazat = [cd_brno, rj_brno, rj_brnova, cd_ova, rj_ova, le_ova, arr_libpa, arr_bud, cd_kv]
len(ukazat)
for u in ukazat:
    u.name = u.name.replace("-"," &#8596; ")
    u.index = u.index.map(lambda x: f"{x} d")

ČD: Praha hl.n.-Brno hl.n.
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 149.0 min.


C:\Users\micha\AppData\Local\Temp\ipykernel_30048\2732313708.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]


Celkem řádků: 17573

RJ: Praha-Brno
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 148.0 min.


C:\Users\micha\AppData\Local\Temp\ipykernel_30048\2732313708.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]


Celkem řádků: 10575

RJ: Brno-Ostrava
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 128.0 min.


...

C:\Users\micha\AppData\Local\Temp\ipykernel_30048\2732313708.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]


Celkem řádků: 14660

RJ: Ostrava-Praha
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 187.0 min.


C:\Users\micha\AppData\Local\Temp\ipykernel_30048\2732313708.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]


Celkem řádků: 8253

LE: Ostrava-Praha
Pozor, vylezlo nám tu -1 přestupů. Opravuji.
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 197.0 min.
Celkem řádků: 9864



C:\Users\micha\AppData\Local\Temp\ipykernel_30048\2732313708.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]


In [233]:
farby = []
for u in ukazat:
    if "ČD" in u.name:
        farby.append("#173a79")
    elif "RJ" in u.name:
        farby.append("#fabb00")
    elif "LE" in u.name:
        farby.append("#010202")
    elif "ARR" in u.name:
        farby.append("#00becd")
farby

['#173a79',
 '#fabb00',
 '#fabb00',
 '#173a79',
 '#fabb00',
 '#010202',
 '#00becd',
 '#00becd',
 '#173a79']

In [241]:
def irozhlas_graf(
    carovy=[],
    sloupcovy=[],
    vodorovny=[],
    rozpeti=[],
    procenta=[],
    skryte=[],
    barvy=[
            "#b2e061",  ## světle zelená (light green)
            "#7eb0d5",  ## světle modrá (light blue)
            "#fd7f6f",  ## světle červená (light red)
            "#bd7ebe",  ## světle fialová (light purple)
            "#ffb55a",  ## oranžová (orange)
            "#ffee65",  ## žlutá (yellow)
            "#beb9db",  ## levandulová (lavender)
            "#fdcce5",  ## skoro černá
            "#8bd3c7",  ## světle tyrkysová (light turquoise),
        "red",
        "blue",
        "purple"
        ],
    histogram=False,
    max_procenta=100,
    target="",
    titulek="",
    podtitulek="",
    naproti=[],
    osay=" ",
    osay2=" ",
    osaymin=None,
    osaymax=None,
    kredity=["zdroj dat a autorstvo", "url odkazu"],
    zaokrouhleni=1,
    prvni=True,
    skladany=False,
    naopak=False,
    vzhurunohama=False,
    skrytnuly=False,
):
    """
    Funkce vygeneruje HighCharts graf z pandas Series (jedné nebo více).

    iROZHLAS-friendly barvy:
    - "#b2e061" světle zelená
    - "#7eb0d5" světle modrá
    - "#fd7f6f" světle červená
    - "#bd7ebe" světle fialová
    - "#ffb55a" oranžová
    - "#ffee65" žlutá
    - "#beb9db" levandulová
    - "#fdcce5" skoro černá
    - "#8bd3c7" světle tyrkysová
    """

    import os
    import pandas as pd
    from highcharts_core.chart import Chart
    from highcharts_core.options.series.area import LineSeries
    from highcharts_core.options.series.bar import ColumnSeries
    from highcharts_core.options.series.bar import BarSeries
    from highcharts_core.options.series.area import AreaRangeSeries
    from highcharts_core.options.series.histogram import HistogramSeries
    from highcharts_core.options.legend import Legend
    from highcharts_core.options.title import Title
    from highcharts_core.options.subtitle import Subtitle
    from highcharts_core.options.credits import Credits

    pocitadlo_barev = 0
    
    nastaveni = {}

    if prvni:
        zdrojaky = f"""<script src="https://code.highcharts.com/highcharts.js"></script><script src="https://code.highcharts.com/highcharts-more.js"></script><script src="https://code.highcharts.com/modules/exporting.js"></script><script src="https://code.highcharts.com/modules/export-data.js"></script><script src="https://code.highcharts.com/modules/accessibility.js"></script>
        <link
  rel="stylesheet"
  href="https://fonts.googleapis.com/css?family=Noticia+Text"
/>
<link
  href="https://fonts.googleapis.com/css2?family=Asap"
  rel="stylesheet"
/>
<link
  href="https://fonts.googleapis.com/css2?family=Roboto"
  rel="stylesheet"
/>
<link
  rel="stylesheet"
  href="https://data.irozhlas.cz/hhighcharts-template/style.css"
/>
<link
  rel="stylesheet"
  href="https://data.irozhlas.cz/hhighcharts-template/highcharts-style.css"
/>
<style type="text/css">text{{font-family:"Asap"!important}}.paragraph{{font-family:"Noticia text"!important}}.href{{color:#666;fill:#666}}.highcharts-title{{font-family:"Noticia text"!important;font-weight:700!important;text-align:left!important;left:10px!important}}.highcharts-subtitle{{text-align:left!important;font-size:.95rem!important;left:10px!important;font-family:"Asap"!important}}.highcharts-data-labels text{{font-size:.85rem!important}}.highcharts-axis-labels text{{font-size:.85rem!important}}text.highcharts-plot-line-label{{font-size:.85rem!important;fill:#666}}text.highcharts-plot-band-label{{font-size:.85rem!important;fill:#666}}text.highcharts-credits{{font-size:.75rem!important}}.highcharts-tooltip span{{font-family:"Asap"!important}}.axis-label-on-tick{{fill:#aaa;color:#aaa}}.mock-empty-line{{fill:#fff;color:#fff}}</style>"""
    else:
        zdrojaky = ""

    pred = f"""{zdrojaky}
        <figure id="{target}">
        <div id="container"></div>
        </figure>
        <script>"""

    if len(carovy) > 0:
        categories = carovy[0].index.to_list()
    if len(sloupcovy) > 0:
        categories = sloupcovy[0].index.to_list()
    if len(vodorovny) > 0:
        categories = vodorovny[0].index.to_list()
    if len(rozpeti) > 0:
        categories = rozpeti[0].index.to_list()

    categories = [str(x) for x in categories]

    nastaveni["xAxis"] = {"categories": categories, "min": 0}
    nastaveni["yAxis"] = [
        {
            "title": {"text": osay},
            "reversed": vzhurunohama,
            "max": osaymax,
            "min": osaymin,
        }
    ]

    if skladany:
        if len(sloupcovy) > 0:
            nastaveni["plotOptions"] = {"column": {"stacking": "normal"}}
        if len(vodorovny) > 0:
            nastaveni["plotOptions"] = {"bar": {"stacking": "normal"}}
    if histogram:
        nastaveni["plotOptions"] = {
            "column": {
                "pointPadding": 0,
                "borderWidth": 0,
                "groupPadding": 0,
                "shadow": False,
            }
        }

    
    if len(procenta) > 0:
        osa_procent = {
            "title": {"text": osay2},
            "max": max_procenta,
            "min": 0,
            "labels": {"format": "{value} %"},
        }

        if len(procenta) != len(carovy) + len(sloupcovy):
            osa_procent["opposite"] = True
            druha_osa = 1
            nastaveni["yAxis"].append(osa_procent)
            nastaveni["alignTicks"] = False
        if len(procenta) == len(carovy) + len(sloupcovy):
            nastaveni["yAxis"] = [osa_procent]
            druha_osa = 0

    if len(naproti) > 0:
        druha_osa = 1
        druha_osa_y = {
            "title": {"text": osay2},
            "opposite": True,
            "max": naproti[0].max(),
            "min": 0,
        }
        nastaveni["yAxis"].append(druha_osa_y)

    my_chart = Chart(container=target, options=nastaveni)

    procenta = [p.name for p in procenta]
    naproti = [n.name for n in naproti]
    skryte = [s.name for s in skryte]

    def vykresleni(serie, typ):

        pocitadlo_barev = 0
        
        for s in serie:
            popisek = s.name
            print(popisek)

            if s.name in skryte:
                viditelnost = False
            else:
                viditelnost = True

            ktera_osa = 0
            if s.name in naproti:
                ktera_osa = druha_osa

            if s.name in procenta:
                s = [round(x * 100, zaokrouhleni) for x in s.fillna(0).to_list()]
                my_chart.add_series(
                    typ(
                        data=s,
                        visible=viditelnost,
                        name=popisek,
                        y_axis=druha_osa,
                        tooltip={"valueSuffix": " %"},
                    )
                )

            if any(s.name == x.name for x in rozpeti):
                line_id = f"line_{popisek}"
                print(line_id)
                my_chart.add_series(
                    LineSeries(
                        data=s.apply(lambda x: x[2]).fillna(0).to_list(),
                        visible=viditelnost,
                        id = line_id,
                        name=popisek,
                        y_axis=ktera_osa,
                        color = barvy[pocitadlo_barev],
                        tooltip={"valuePrefix": "střední: ","valueSuffix": " Kč"},
                    ))
                my_chart.add_series(
                    AreaRangeSeries(
                        data=s.apply(lambda x: x[0:2]).fillna(0).to_list(),
                        visible=viditelnost,
                        type='arearange',
                        name=popisek,
                        linkedto = line_id,
                        y_axis=ktera_osa,
                        color = barvy[pocitadlo_barev],
                        fillOpacity=0.3,
                        tooltip={"valueSuffix": " Kč"},
                    lineWidth=0,
                    marker={"enabled": False}
                    ))
                pocitadlo_barev += 1

   #         else:
                #my_chart.add_series(
                    #typ(
                     #   data=s.fillna(0).to_list(),
                    #    visible=viditelnost,
                   #     name=popisek,
                  #      y_axis=ktera_osa,
                 #   )
                #)

    if len(sloupcovy) > 0:
        vykresleni(sloupcovy, ColumnSeries)
    if len(carovy) > 0:
        vykresleni(carovy, LineSeries)
    if len(vodorovny) > 0:
        vykresleni(vodorovny, BarSeries)
    if len(rozpeti) > 0:
        vykresleni(rozpeti, AreaRangeSeries)

    my_chart.options.colors = barvy

    if naopak:
        my_chart.options.legend = Legend(reversed=True)

    my_chart.options.title = Title(text=titulek, align="left", margin=30)
    
    if len(podtitulek) > 0:
        my_chart.options.subtitle = Subtitle(text=podtitulek, align="left")

    my_chart.options.credits = Credits(text=kredity[0], enabled=True, href=kredity[1])
    my_chart.options.height = 800

    as_js_literal = my_chart.to_js_literal()

    if skrytnuly == True:
        as_js_literal = as_js_literal.replace("y: 0.0", "y: null")

 #   if len(rozpeti) > 0:
    
  #      as_js_literal = as_js_literal.replace(
   #         '"type":"arearange"', 
    #        '"type":"arearange", "linkedTo": "previous", "fillOpacity": 0.3, "lineWidth": 0, "marker": {"enabled": false}'
     #   )

    
#    if len(rozpeti) > 0:
#        as_js_literal = as_js_literal.replace("type: 'arearange'", "type: 'arearange',linkedTo: 'previous',fillOpacity: 0.3,lineWidth: 0,marker: {enabled: false}")

    as_js_literal = as_js_literal.splitlines()

    as_js_literal2 = []
    
    for line in as_js_literal:
        if 'id: ' in line:
            ajdy = line.split("'")[1]
            print(ajdy)
        if "type: 'arearange'" in line:
            line = line.replace(" type: 'arearange'",f" type: 'arearange',linkedTo: '{ajdy}', fillOpacity: 0.3, lineWidth: 0")
        as_js_literal2.append(line)
    as_js_literal2 = "\n".join(as_js_literal2)
    
    code = f"<html><head><title>{titulek}</title></head><body>{pred}{as_js_literal2}</script></body></html>"

    
    if not os.path.exists("grafy"):
        os.mkdir("grafy")

    with open(os.path.join("grafy", target + ".html"), "w+") as f:
        f.write(code)

    with open(os.path.join("grafy", target + ".txt"), "w+") as f:
        f.write(f"{pred}{as_js_literal2}</script>")

        print("Graf uložen.")

In [243]:
irozhlas_graf(rozpeti=ukazat, skryte=ukazat[1:], barvy=farby, target='vnitrostatni_jizdne', titulek='Vnitrostátní jízdné podle předstihu nákupu', osaymin=0, podtitulek='Nejnižší, střední a nejvyšší cena na přímých spojích. Data zahrnují pouze spoje vyjíždějící před změnami jízdního řádu a částečným zdražením 15. 12.', kredity=['Zdroj dat: e-shopy dopravců od 7. 11. do 1. 12. 2024. Vizualizace: iROZHLAS.cz','https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika'])

ČD Praha &#8596; Brno
line_ČD Praha &#8596; Brno
RJ Praha &#8596; Brno
line_RJ Praha &#8596; Brno
RJ Brno &#8596; Ostrava
line_RJ Brno &#8596; Ostrava
ČD Praha &#8596; Ostrava
line_ČD Praha &#8596; Ostrava
RJ Praha &#8596; Ostrava
line_RJ Praha &#8596; Ostrava
LE Praha &#8596; Ostrava
line_LE Praha &#8596; Ostrava
ARR Liberec &#8596; Pardubice
line_ARR Liberec &#8596; Pardubice
ARR Praha &#8596; Budějovice
line_ARR Praha &#8596; Budějovice
ČD Ostrava &#8596; Karlovy Vary
line_ČD Ostrava &#8596; Karlovy Vary
line_ČD Praha &#8596; Brno
line_RJ Praha &#8596; Brno
line_RJ Brno &#8596; Ostrava
line_ČD Praha &#8596; Ostrava
line_RJ Praha &#8596; Ostrava
line_LE Praha &#8596; Ostrava
line_ARR Liberec &#8596; Pardubice
line_ARR Praha &#8596; Budějovice
line_ČD Ostrava &#8596; Karlovy Vary
Graf uložen.
